In [ ]:
# %pip install pandas
# %pip install geopandas

CSV data prepared from California Bureau of Public Health, Data on Suicide and Self Harm, 2018-2020. https://www.cdph.ca.gov/Programs/CCDPHP/DCDIC/SACB/Pages/Data-on-Suicide-and-Self-Harm.aspx (Suicide rates by county table)

In [11]:
import pandas as pd
import geopandas as gpd

geopackage_name = 'output/california_suicides.gpkg'

In [ ]:
gdf = gpd.read_file('/Users/ecopo/GeospatialResources/NaturalEarth/Counties/ne_10m_admin_2_counties/ne_10m_admin_2_counties.shp')

In [ ]:
def dissolve_counties(gdf, counties):
    dissolved = gdf[gdf['NAME'].isin(counties)].dissolve()
    dissolved['NAME'] = ', '.join(counties)
    return dissolved

california_counties = gdf[gdf['REGION'] == 'CA'][['NAME', 'geometry']]

northern_merge = ['Del Norte', 'Lassen', 'Modoc', 'Plumas', 'Sierra', 'Siskiyou', 'Trinity']
north_central_merge = ['Colusa', 'Glenn', 'Tehama']
southern_merge = ['Alpine', 'Amador', 'Calaveras', 'Inyo', 'Mariposa', 'Mono', 'Tuolumne']

northern_dissolved = dissolve_counties(california_counties, northern_merge)
north_central_dissolved = dissolve_counties(california_counties, north_central_merge)
southern_dissolved = dissolve_counties(california_counties, southern_merge)

unmerged_counties = california_counties[~california_counties['NAME']
                                        .isin(northern_merge + north_central_merge + southern_merge)]
merged_counties = pd.concat([northern_dissolved, north_central_dissolved, southern_dissolved])
original_counties_that_were_merged = california_counties[california_counties['NAME'].isin(northern_merge + north_central_merge + southern_merge)]

all_counties_with_merged = pd.concat([unmerged_counties, merged_counties])

california_counties.to_file(geopackage_name, layer='california_counties', driver='GPKG')
merged_counties.to_file(geopackage_name, layer='merged_counties', driver='GPKG')
all_counties_with_merged.to_file(geopackage_name, layer='all_counties_with_merged', driver='GPKG')
original_counties_that_were_merged.to_file(geopackage_name, layer='original_counties_that_were_merged', driver='GPKG')
unmerged_counties.to_file(geopackage_name, layer='unmerged_counties', driver='GPKG')